## Import Libraries

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.regression import LSTM

## PySpark Session

In [ ]:
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

## Load data

In [ ]:
data = spark.read.format("csv").option("header", "true").load("my_file.csv")

## Create feature vector [f1, f2, f3] as input of model

In [ ]:
featureColumns = ["f1", "f2", "f3"]
assembler = VectorAssembler(inputCols=featureColumns, outputCol="features")
data = assembler.transform(data)

## Split data into train and test

In [ ]:
(trainingData, testData) = data.randomSplit([0.8, 0.2])

## Model Initialization 

In [ ]:
lstm = LSTM(featuresCol="features", labelCol="target", predictionCol="prediction",
maxIter=100, blockSize=64, stepSize=0.1, seed=0)

## Create parameter grid for hyperparameter tuning

In [ ]:
paramGrid = ParamGridBuilder()
.addGrid(lstm.maxIter, [100, 200])
.addGrid(lstm.blockSize, [64, 128])
.addGrid(lstm.stepSize, [0.1, 0.01]).build()

## Create evluator for regression performance

In [ ]:
evaluator = RegressionEvaluator(labelCol="target", metricName="rmse")

## Create cross-validation model 

In [ ]:
crossValidator = CrossValidator(estimator=lstm, estimatorParamMaps=paramGrid, evaluator=evaluator)

In [ ]:
## 